In [ ]:
#this code is for Julia. This calculated the theoretical maximum output of 1 process given multiple constraints. JuMP is the main package for this.
#import relevant package for Julia.
using JuMP
using Cbc
using XLSX
using CSV
using DataFrames
using Glob

In [ ]:
#call for a model with Cbr optimizer
m = Model(Cbc.Optimizer);
set_optimizer_attribute(m, "logLevel", 0) #set solver attribute. Not necessary, as constraint do not have non-linear relations.

#all a,b,c, with numbers are a constant typed in. 
#current condition of machine
Me = [a1,b1,...e1]
S = [a2,b2,...e2]
...
ini_rat1 = [a6,b6,...e6]
#maximum limit/constraint of machine
lim_Me = [lim_a1,lim_a2,...]
lim_S = [lim_b1,lim_b2,...]
lim_MeO = a.*[1,1,1,1,1,1];
lim_BarP = b .*[1,1,1,1,1,1];

println(size(lim_Me,1))

In [ ]:
@variables(m, begin #create variables mp_sd, s_sd, where consstraint between 0 < current cond < constraint cond
        #lim
    0 <= mp_sd[i = 1:size(Me,1)] <= lim_Me[i];
    0 <= s_sd[i = 1:size(S,1)] <= lim_S[i];
    ...
    0 <= B_P[i = 1:size(BarP,1)] <= lim_BarP[i];
    
    PPH_fin[i= 1:size(PPH_cur,1)] >= 0; #variables with ratio constraint, hence between 0<1. 
    0 <= extr_rat[i =1:6] <= 1;
    -1<= rat_del[i = 1:6] <= 1;
    PPH1;
    PPH2;
    max_PPH; #these are calculate variables with no constraint.
    end)

In [64]:
@constraints m begin #create constraints for all variables
    sum(extr_rat[1:3]) == 1; #sum of all ratio = 1
    sum(extr_rat[4:6]) == 1; #sum of all ratio = 1
    0.5*sum(extr_rat) == 1;
    ini_rat1[1] .+ rat_del[1] == extr_rat[1]; #A #process part constraint, where initial ratio + extra ratio = final ratio
    ini_rat1[2] .+ rat_del[2] == extr_rat[2]; #B
    ini_rat1[3] .+ rat_del[3] == extr_rat[3]; #C
    ini_rat1[4] .+ rat_del[4] == extr_rat[4]; #D
    ini_rat1[5] .+ rat_del[5] == extr_rat[5]; #E
    ini_rat1[6] .+ rat_del[6] == extr_rat[6]; #F
    mp_sd./melt_pump_sd .== extr_rat./ini_rat1; #ratio of extr_rat calculated as such. scaling current condition by how much will change ratio by certain amount.
    PPH_fin./PPH_cur .== extr_rat./ini_rat1;
    s_sd./screw_sd .== extr_rat./ini_rat1;
    mpo_P./melt_pump_outlet_P .== extr_rat./ini_rat1;
    B_P./Barrel_P .== extr_rat./ini_rat1;
    PPH1 .== sum(PPH_fin[1:3]); # sum up productions 
    PPH2 .== sum(PPH_fin[4:6]);
    max_PPH .== PPH1 + PPH2; #sum up productions of both machines
end

@objective(m, Max, max_PPH); #object is to maximize production ppH as max_PPH.

optimize!(m)

Presolve 1 (-41) rows, 3 (-42) columns and 3 (-92) elements
0  Obj 5764.3465 Primal inf 1031.7614 (1) Dual inf 2.9468086 (3)
1  Obj 6802.6225
Optimal - objective value 6802.6225
After Postsolve, objective 6802.6225, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 6802.622471 - 1 iterations time 0.002, Presolve 0.00


In [ ]:
print(value.(extr_rat))
print(value.(sum(extr_rat[1:3])))
print(value.(sum(extr_rat[4:6])))

In [ ]:
print(value.(rat_del))

In [ ]:
print(value.(mp_sd))

In [ ]:
print(melt_pump_sd)

In [ ]:
print(value.(extr_rat./ini_rat1))

In [ ]:
value.(mp_sd./melt_pump_sd)

In [ ]:
value.(PPH_fin)

In [ ]:
PPH_cur

In [ ]:
value.(B_P)

In [ ]:
value.(mpo_P)

In [ ]:
value.(mpo_P./melt_pump_outlet_P)

In [ ]:
value.(sum(PPH_fin[1:3]))

In [ ]:
value.(sum(PPH_fin[4:6]))

In [ ]:
value.(max_PPH)

In [ ]:
sum(PPH_cur)